In [13]:
import math
import pandas as pd 
import numpy as np


data = pd.read_csv('cat4.csv')


# print(data.head())


#features
x = data.drop(['index', 'galex_objid', 'sdss_objid', 'class', 'spectrometric_redshift', 'pred'], axis = 1) 
# print(x.head())


#class
y = data['class']

redshift = data['spectrometric_redshift']
# print(y.head())

In [14]:
def sigmoid(x):
    return 1/(1 + np.exp(-x))



#fix scientifci notations to normal floats
# def scitofloat(x):
#     return x

In [15]:
#normalise the values of x

outercnt = 0
for index, row in x.iterrows():
    innercnt = 0
    for column in x:
#         x.at[outercnt, column] = scitofloat(x.iloc[outercnt, innercnt])
        x.at[outercnt, column] = sigmoid(x.iloc[outercnt, innercnt])
#         print(outercnt, column, '\t\t', x.iloc[outercnt, innercnt])
        innercnt += 1
    outercnt += 1    

#     print('\n\n')

print(x.head())

     u    g    r    i    z  extinction_u  extinction_g  extinction_r  \
0  1.0  1.0  1.0  1.0  1.0      0.518859      0.514697      0.510169   
1  1.0  1.0  1.0  1.0  1.0      0.520498      0.515976      0.511054   
2  1.0  1.0  1.0  1.0  1.0      0.520396      0.515896      0.510999   
3  1.0  1.0  1.0  1.0  1.0      0.520920      0.516305      0.511282   
4  1.0  1.0  1.0  1.0  1.0      0.520923      0.516307      0.511283   

   extinction_i  extinction_z  ...       u-g       u-r       u-i       u-z  \
0      0.507557      0.505621  ...  0.563752  0.639445  0.699413  0.633595   
1      0.508215      0.506111  ...  0.511422  0.494797  0.532386  0.563918   
2      0.508174      0.506080  ...  0.627986  0.682746  0.648898  0.701978   
3      0.508384      0.506237  ...  0.582493  0.647692  0.726401  0.764159   
4      0.508385      0.506237  ...  0.506167  0.463440  0.414806  0.390012   

        g-r       g-i       g-z       r-i       r-z       i-z  
0  0.578484  0.642929  0.572306  0

In [16]:
def info_dataset(data, verbose=True):
    label1, label2 = 0, 0
    data_size = len(data)
    for index, row in data.iterrows():
        label = row['class']
#         print(label)
        if label == 1:
            label1 += 1
        else:
            label2 += 1
    if verbose:
        print('Total of samples: %d' % data_size)
        print('Total label 1: %d' % label1)
        print('Total label 0: %d' % label2)
    return [len(data), label1, label2]

In [17]:
p = 0.6
_, label1, label2 = info_dataset(data,False)

info_dataset(data)

Total of samples: 33463
Total label 1: 23389
Total label 0: 10074


[33463, 23389, 10074]

In [18]:
#get training set and test set

from sklearn.model_selection import train_test_split

xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size = 0.2, random_state = 15)

train_set = pd.concat([xTrain, yTrain], axis=1, sort=False)
test_set = pd.concat([xTest, yTest], axis=1, sort=False)


print(train_set.iloc[0:4])
# print(train_set.iloc[0][1])

         u    g    r    i    z  extinction_u  extinction_g  extinction_r  \
17141  1.0  1.0  1.0  1.0  1.0      0.563760      0.549788      0.534503   
31720  1.0  1.0  1.0  1.0  1.0      0.513283      0.510351      0.507161   
30303  1.0  1.0  1.0  1.0  1.0      0.512217      0.509520      0.506586   
25396  1.0  1.0  1.0  1.0  1.0      0.531213      0.524334      0.516841   

       extinction_i  extinction_z  ...       u-r       u-i       u-z  \
17141      0.525657      0.519092  ...  0.582326  0.564625  0.525910   
31720      0.505322      0.503959  ...  0.438573  0.473958  0.515487   
30303      0.504895      0.503641  ...  0.622254  0.627557  0.692209   
25396      0.512517      0.509311  ...  0.702868  0.761691  0.794556   

            g-r       g-i       g-z       r-i       r-z       i-z  class  
17141  0.569140  0.551310  0.512433  0.481914  0.443098  0.461023      1  
31720  0.466001  0.501621  0.543072  0.535612  0.576623  0.541463      1  
30303  0.500226  0.505882  0.5774

In [19]:
def euclidian_dist(p1, p2):
    dim, sum_ = len(p1), 0
    for index in range(dim - 1):
        sum_ += math.pow(p1[index] - p2[index], 2)
    return math.sqrt(sum_)

In [20]:
def knn(train_set, new_sample, K):

#     print('train set', train_set, type(train_set))
#     print('new sample', new_sample, type(new_sample))
    dists, train_size = {}, len(train_set)

    for i in range(train_size):
#         print(train_set.iloc[i-1].values.tolist())
#         print(new_sample.values.tolist()[0])
#         print('\n\n')
        d = euclidian_dist(train_set.iloc[i-1].values.tolist(), new_sample.values.tolist()[0])
        dists[i] = d
        
#     print('done')
    
    k_neighbors = sorted(dists, key=dists.get)[:K]
    
    qty_label1, qty_label2 = 0, 0
    
    for index in k_neighbors:
#         print('index, ', index)
#         print(train_set.iloc[index])
        
    
        #fix this
        if train_set.iloc[index].values.tolist()[-1] == 1:
            qty_label1 += 1
        else:
            qty_label2 += 1
        
            
    if qty_label1 > qty_label2:
        return 1
    else:
        return 0

In [27]:
correct, K = 0, 3


count, tp, fp, tn, fn = 0, 0, 0, 0, 0


for index, sample in test_set.iterrows():
    print(count)
    count +=1
    if count == 200:
        break
    sample = pd.DataFrame(sample)
    sample = sample.T

    label = knn(train_set, sample, K)

    if label == sample.iloc[0].values.tolist()[-1]:
        correct += 1
    
    print('predicted, actual', label, sample.iloc[0].values.tolist()[-1])
    if label == 1 and sample.iloc[0].values.tolist()[-1] == 1:
        tp += 1
    if label == 1 and sample.iloc[0].values.tolist()[-1] == 0:
        fp += 1 
    if label == 0 and sample.iloc[0].values.tolist()[-1] == 1:
        fn += 1    
    if label == 0 and sample.iloc[0].values.tolist()[-1] == 0:
        tn += 1    
        

print('total: ', count)
print('correct: ', correct)
print('\n\n')

matrix_list = np.array([[tp, fp], [fn, tn]])

# matrix = pd.DataFrame({'Actual +ve': matrix_list[:, 0], 'Actual -ve': matrix_list[:, 1]})
matrix = pd.DataFrame(matrix_list, index=['Pred +ve', 'Pred -ve'], columns=['Actual +ve', 'Actual -ve'])



print(matrix)

0
predicted, actual 1 1.0
1
predicted, actual 1 0.0
2
predicted, actual 1 1.0
3
predicted, actual 1 1.0
4
predicted, actual 1 1.0
5
predicted, actual 1 1.0
6
predicted, actual 1 1.0
7
predicted, actual 1 0.0
8
predicted, actual 1 1.0
9
predicted, actual 1 1.0
10
predicted, actual 1 0.0
11
predicted, actual 1 1.0
12
predicted, actual 1 1.0
13
predicted, actual 1 1.0
14
predicted, actual 1 1.0
15
predicted, actual 1 1.0
16
predicted, actual 1 1.0
17
predicted, actual 1 1.0
18
predicted, actual 1 1.0
19
predicted, actual 1 1.0
20
predicted, actual 1 1.0
21
predicted, actual 1 1.0
22
predicted, actual 1 1.0
23
predicted, actual 0 0.0
24
predicted, actual 0 1.0
25
predicted, actual 1 1.0
26
predicted, actual 1 1.0
27
predicted, actual 1 0.0
28
predicted, actual 1 0.0
29
predicted, actual 1 0.0
30
predicted, actual 1 1.0
31
predicted, actual 0 0.0
32
predicted, actual 1 1.0
33
predicted, actual 1 1.0
34
predicted, actual 1 1.0
35
predicted, actual 1 1.0
36
predicted, actual 1 1.0
37
predicte

In [26]:
print("Train set size: %d" % len(train_set))
print("Test set size: %d" % count)
print("Correct predicitons: %d" % correct)
print("Accuracy: %.2f%%" % (100 * correct / count))

precision = tp/(tp + fp)
recall = tp/(tp + fn)
fscore = 2*((precision * recall)/(precision + recall))

print('precision: ', precision)
print('recall: ', recall)
print('fscore: ', fscore)

Train set size: 26770
Test set size: 50
Correct predicitons: 39
Accuracy: 78.00%
precision:  0.8181818181818182
recall:  0.9473684210526315
fscore:  0.8780487804878049
